In [2]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.python.keras.optimizers import Adadelta, Nadam
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.utils import multi_gpu_model, plot_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.python.keras.preprocessing import image
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
import cv2
from multiclassunet import Unet
from dilatednet import DilatedNet
import tqdm
import time
from tensorflow.python.keras.utils import Sequence
from moviepy.editor import VideoFileClip, ImageSequenceClip

In [2]:
unet = Unet(256, 256, 8, 64)
# unet = DilatedNet(256, 256, 8, True, True)
# unet.load_weights('dilated.h5')
p_unet = multi_gpu_model(unet, 4)
# unet.load_weights('unet1m.h5')
p_unet.load_weights('models-dr/unet50k.h5')

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [ ]:
# s = time.time()
# for i in tqdm.tqdm(range(len(image_list))):
#     img = image.load_img(f'{test_dir}/{image_list[i]}')
#     x = image.img_to_array(img)/255.
#     img = image.img_to_array(img)
#     _img = img.copy()
#     dims = x.shape
#     x = cv2.resize(x, (256, 256))
    
#     z = unet.predict(np.expand_dims(x, axis=0))
#     z = np.squeeze(z)
#     z = z.reshape(256, 256, 8)
#     z = cv2.resize(z, (dims[1], dims[0]))
#     y = np.argmax(z, axis=2)
#     y = np.argmax(z, axis=2)
    
#     construction = np.zeros_like(y)
#     human = np.zeros_like(y)
#     vehicle = np.zeros_like(y)
    
#     construction[y==2] = 255.
#     human[y==6] = 255.
#     vehicle[y==7] = 255.

# #     fig = plt.figure(figsize=(40, 10))
# #     ax = fig.add_subplot(1,2,1)
# #     ax.set_title(f'Image')
# #     ax.imshow(img/255.)
    
#     result = img.copy()
#     alpha = 0.3
#     img[:,:,1] = construction
#     img[:,:,2] = vehicle 
#     img[:,:,0] = human


#     result = cv2.addWeighted(img, alpha, result, 1-alpha, 0, result)
# #     ax3 = fig.add_subplot(1,2,2)
# #     ax3.set_title(f'Generated_Mask {i}')
# #     ax3.imshow(result/255.)
    
#     cv2.imwrite(f'outputs/stuggart_02/{image_list[i]}', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
# #     cv2.imwrite(f'outputs/images/{i}.png', cv2.cvtColor(_img, cv2.COLOR_RGB2BGR))

# #     plt.close(fig)
    
# e = time.time()
# t = str(e-s)
# o = str((e-s)/len(image_list))
# print(f'total time taken = {t}')
# print(f'time taken per image = {o}')

In [ ]:
clip = VideoFileClip('test4k.mp4',target_resolution=(288, 512))
# res = clip.fl_image(pipeline)
res.write_videofile('0.mp4', audio=False)

In [ ]:
%%HTML
<video width="512" height="288" controls>
  <source src="outputs/s0.mp4" type="video/mp4">
</video>

In [ ]:
def pp(image):
    alpha = 0.6
    dims = image.shape
    x = cv2.resize(image, (256, 256))
    x = np.float32(x)/255.
    z = unet.predict(np.expand_dims(x, axis=0))
    z = np.squeeze(z)
    z = z.reshape(256, 256, 8)
    z = cv2.resize(z, (dims[1], dims[0]))
    y = np.argmax(z, axis=2)
    
    img_color = image.copy()   
    for i in range(dims[0]):
        for j in range(dims[1]):
            img_color[i, j] = color_map[str(y[i, j])]
    cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)
    return img_color

In [ ]:
i = 9
test = image.load_img(f'{test_dir}/{image_list[i]}')
test = image.img_to_array(test)
result = pp(test)
plt.imshow(result/255.)

In [49]:
test_dir = 'validate/stuttgart_02/'
image_list = os.listdir(test_dir)
print(f'{len(image_list)} frames found')

1200 frames found


In [ ]:
color_map = {
'0': [0, 0, 0],
 '1': [153, 153, 0],
 '2': [255, 204, 204],
 '3': [255, 0, 127],
 '4': [0, 255, 0],
 '5': [0, 204, 204],
 '6': [255, 0, 0],
 '7': [0, 0, 255]
}

s = time.time()
for t in tqdm.tqdm(range(len(image_list))):
    img = image.load_img(f'{test_dir}/{image_list[t]}')
    img = image.img_to_array(img)
    alpha = 0.6
    dims = img.shape
    x = cv2.resize(img, (256, 256))
    x = np.float32(x)/255.
    z = unet.predict(np.expand_dims(x, axis=0))
    z = np.squeeze(z)
    z = z.reshape(256, 256, 8)
    z = cv2.resize(z, (dims[1], dims[0]))
    y = np.argmax(z, axis=2)
    
    img_color = img.copy()   
    for i in range(dims[0]):
        for j in range(dims[1]):
            img_color[i, j] = color_map[str(y[i, j])]
    cv2.addWeighted(img, alpha, img_color, 1-alpha, 0, img_color)
    cv2.imwrite(f'outputs/stuggart02/{image_list[t]}', cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR))   
e = time.time()
t = str(e-s)
o = str((e-s)/len(image_list))
print(f'total time taken = {t}')
print(f'time taken per image = {o}')

In [3]:
for folder in ['stuggart00', 'stuggart01', 'stuggart02']:
    print(f'loading video {folder}')
    clip = ImageSequenceClip(f'outputs/{folder}/', fps=18, load_images=True)
    print(f'Successfully loaded video {folder}')
    clip.write_videofile(f'outputs/videos/{folder}.mp4')
    print(f'Successfully created video {folder}')

loading video stuggart00
Successfully loaded video stuggart00
[MoviePy] >>>> Building video outputs/videos/stuggart00.mp4
[MoviePy] Writing video outputs/videos/stuggart00.mp4


100%|██████████| 600/600 [00:26<00:00, 22.90it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputs/videos/stuggart00.mp4 

Successfully created video stuggart00
loading video stuggart01
Successfully loaded video stuggart01
[MoviePy] >>>> Building video outputs/videos/stuggart01.mp4
[MoviePy] Writing video outputs/videos/stuggart01.mp4


100%|██████████| 1101/1101 [00:50<00:00, 21.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputs/videos/stuggart01.mp4 

Successfully created video stuggart01
loading video stuggart02
Successfully loaded video stuggart02
[MoviePy] >>>> Building video outputs/videos/stuggart02.mp4
[MoviePy] Writing video outputs/videos/stuggart02.mp4


100%|██████████| 1201/1201 [00:52<00:00, 22.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputs/videos/stuggart02.mp4 

Successfully created video stuggart02
